# Lab 1
CMSC 457 Spring 2021  
Prepared by Ethan Hickman (ethanh \[at\] umd.edu)

## What we're going to cover
- Brief overview of quantum programming languages
- What Qiskit offers and the Qiskit stack
- Jupyter speedrun
- A few quantum protocols:
    - Superdense coding
    - Deutsch algorithm
    - Classical query complexity of the Deutsch-Josza algorithm

## Quantum programming languages — an incomplete list
See 'Quantum Programming' on Wikipedia for a more complete list and links to each.

### Quantum assembly
- cQASM: "common QASM," for __hardware agnostic__ intermediate representation (IR)
- OpenQASM (IBM): IR for IBM devices
- Blackbird (Xanadu): IR for Xanadu's photonic devices
- Is this really assembly? (What will devices look like later?)

### Mid-level languages
- Qiskit (IBM); superconducting
- Forest (Rigetti); superconducting
- Ocean (D-Wave); adiabatic

#### Simulator-only
- Strawberry Fields (Xanadu); photonic
- Cirq (Google); superconducting

### Higher level languages
- Imperative vs. Functional
- Q# (Microsoft): Has quantum "Katas" to help learn
- Silq (ETH): Automatic uncomputation makes programs considerably more concise

## Qiskit

### Python interface

In [ ]:
from qiskit import *
from qiskit.visualization import *
import qiskit

qiskit.__version__

### The four elements 
[IBM docs link](https://qiskit.org/documentation/the_elements.html#terra) with more details and precise language.

[Qiskit main docs](https://qiskit.org/documentation/index.html), where things are split up into these four sections.

#### Terra
The bridge from Qiskit to the quantum hardware.
- Experiment setup
    - Where you actually program the sequence of instructions
    - Gates or control pulses directly to the qubits
    - 'Transpilation' – the problem of mapping a circuit to a real device
- Device interaction
    - Communication with IBM cloud
    - Fair queueing system
    - Async job and results API
    
#### Aer
Simulated backends so we can rapidly test and debug small circuits or circuit elements.
- QasmSimulator
    - Ideal and noisy 'shot' simulation (remember: most quantum protocols are __not__ deterministic! Also, noise makes even deterministic protocols non-deterministic in real life.)
    - A few different simulation techniques to handle some circuit classes more efficiently.
- StatevectorSimulator
    - Calculates the full statevector, with dimension exponential in the number of qubits.
- UnitarySimulator
    - Instead of running your circuit, it tells you what the full unitary of your circuit is. This can be expensive to store, because in the worst case, it requires the square of dimension of the Hilbert space!
    
#### Ignis
- Noise, N̷o̵i̵s̷e̷, N̶̗̯̍ö̸̝͉i̴̫͂̈́s̶͓͘ē̵̱͜
- We're always computing in the presence of noise with analog devices. With quantum devices, this means computing on garbage and leads to sad grad students.
- Circuits
    - Template circuit experiments to characterize the intensity and types of noise in a real device.
    - "Poke it with a stick and see what happens"
- Fitters
    - Analysis on what happened with the Ignis circuits
    - "Postmortem on poking it with the stick"
- Filters
    - Fitter analysis $\rightarrow$ error mitigation techniques

#### Aqua
- (A)lgorithms for (QU)antum (A)pplications
    - Modules for chemistry, AI/ML, optimization, finance
    

## Jupyter, but really fast

In [ ]:
print("This is Python code in a cell, it gets executed when the cell runs")
"The last thing executed in a cell is 'output.'"

In [ ]:
Out  # and Jupyter remembers it!

In [ ]:
assert not (9 + 10 == 21)
assert 9 + 10 == 21, "duH"

In [ ]:
foo = 10

In [ ]:
foo + 2

In [ ]:
foo -= 100

In [ ]:
def add1(x):
    return x + 1

In [ ]:
add1(foo)

In [ ]:
from collections import Counter

import math
import numpy as np

import pprint as pp

rng = np.random.default_rng()

## Working with circuits

A `QuantumCircuit` has a register of qubits and a register of classical bits for measurement results. 

In [ ]:
# Declare a quantum circuit
# Give it either two registers, 
# one +int for the number of qubits in the quantum register
# two +ints for the number of qubits and bits in the quantum/classical registers

qc = qiskit.QuantumCircuit(3, 3)
qc.i(range(3))
qc.sx([0, 2])
qc.y(1)
qc.s(2)

print(qc)

In [ ]:
state = quantum_info.DensityMatrix.from_instruction(qc)

plot_bloch_multivector(state, title="Deutsch Algorithm final state")
# plot_state_city(state, color=['orange', 'cyan'], title="Deutsch Algorithm final state")
# plot_state_hinton(state, title="Deutsch Algorithm final state")

In [ ]:
qc.cx(0,1)
qc.cx(2,1)

qc.measure(0, 0)
qc.measure(1, 1)

qc.draw()

### Some potential rough patches with quantum programming
- ~~No 'intermediate' or partial measurement on the actual IBM devices~~ This actually just got added to the API recently!
- Limited 'quantum volume' of real experiments
    - Defined as $2^n$ for the largest $n$ that doesn't give garbage for 'square' circuits
- Debugging can be _really_ tough.
    - Depends heavily on analysis and visualization tools and how well you plan to test
    - The information is diffuse in the system
    - Noise + a lot of steps in the pipeline between your keyboard and the device
- Most of the API is stable
    
###  Some great upsides to all quantum programming languages/toolkits
- Most of the API is stable
- Interactive, and you can actually just go and try stuff
- Access to literally the newest in quantum computing research

---
## Superdense coding

![](Superdense_coding.png)

We're going to use four entangled pairs of qubits to relay one byte of data at a time. We'll be able to decode $n$ bytes ($8n$ bits) by 'sending' $4n$ qubits! (We can't really send qubits in a simulator, but on way we can emulate it is by only giving Alice access to her allocation of qubits and separately initializing the 'hidden' state again for Bob.)

Alice wants to send Bob an email, but also has a 2 billion dollar research budget.
How can Alice spend as much money as possible in the name of efficiency? With superdense coding!

First, Alice hires an undergrad named Charlie to press literally two buttons (on demand, this is where the big bucks are) to entangle pairs of qubits. 

Charlie carefully packs one qubit from each pair up for Bob, taking care not to drop it* or let it decohere. Alice gets the other qubit, and secretly applies one of $I$, $Z$, $X$, or $ZX$ to that qubit, then Alice sends the modified qubit to Bob too.

Bob has the qubits from both Charlie and Alice now, and can decode the email with a team of post-docs and liberal application of Bell measurement.

*Yt is actually shockingly inexpensive

In [ ]:
# Set a mapping for qubits to give to Alice and Bob
to_bob = [2*i for i in range(4)]
to_alice = [2*i+1 for i in range(4)]

# to_alice = [2*i for i in range(4)]
# to_bob = [2*i+1 for i in range(4)]

# to_alice = [1, 2, 5, 3]
# to_bob = [7, 4, 0, 6]


# Charlie is our entanglement source.
# Press the button, Charlie.
def charlie(qc, to_alice, to_bob):
    qc.reset(range(8))
    qc.h(to_alice)
    qc.cx(to_alice, to_bob)    


# Alice encodes byte by byte, four 'crumbs' at a time.
# (a crumb is a pair of bits; not quite a nibble and much less than a byte)
def alice(qc, byte, to_alice):
    # This byte has the bits ordered 'n(n-1)...3210'
#     print(f"{byte:08b}")

    # A crumb is the word for 2 bits!
    # Given a big-endian byte, we want little-endian list of crumbs.
    # Grab the bits in zero padded pairs with a combo of shift 
    #   and bitwise AND.
    
    # This list will have the bits in pairs ['10', '32', ..., 'n(n-1)']
    crumbs = ['11','10','01','00']
    
    # cidx for 'crumb index'
    for cidx in range(4):
        if crumbs[cidx] == "00":
            pass
        if crumbs[cidx] == "01":
            qc.x(to_alice[cidx])
        if crumbs[cidx] == "10":
            qc.z(to_alice[cidx])
        if crumbs[cidx] == "11":
            qc.z(to_alice[cidx])
            qc.x(to_alice[cidx])

            
def bob(qc, to_alice, to_bob):
    qc.cx(to_alice, to_bob)
    qc.h(to_alice)
    

# Write the quantum circuit
qc = qiskit.QuantumCircuit(8)

charlie(qc, to_alice, to_bob)
qc.barrier()
print("Circuit after Charlie:")
print(qc)

# Test all four crumbs, 00 01 10 11, at once
b = bytearray([0b00011011])
alice(qc, b[0], to_alice)
qc.barrier()
print("Circuit after Alice:")
print(qc)

bob(qc, to_alice, to_bob)
# measure_all applies a barrier()
qc.measure_all()
print("Circuit after Bob:")
print(qc)

In [ ]:
# Noiseless simulation
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1000)
result = job.result()
print("Counts:", result.get_counts(qc))

In [ ]:
res = list(result.get_counts(qc))[0]
print(res)

permutation = list(zip(to_alice, to_bob))[::-1]
print("permutation:", permutation)

# string index = 7-qubit index 
reconstructed_crumbs = [''.join([res[7-qidx] for qidx in tup]) for tup in permutation]
print("crumbs:", reconstructed_crumbs)

reconstructed_byte = ''.join(reconstructed_crumbs)
print("reconstructed:", reconstructed_byte)


In [ ]:

states_from_alice = []
msg = """Hi, Bob!

  Lovely weather today.

From,
Alice"""


backend = Aer.get_backend('statevector_simulator')

byte_arr = bytearray()
byte_arr.extend(map(ord, msg))


for byte in byte_arr:
    qc = QuantumCircuit(8)
    
    charlie(qc, to_alice, to_bob)
    alice(qc, byte, to_alice)
    
    job = execute(qc, backend)
    result = job.result()
    hidden_state = result.get_statevector()
    states_from_alice.append(hidden_state)

backend = Aer.get_backend('qasm_simulator')
decoded = []
for state in states_from_alice:
    qc = qiskit.QuantumCircuit(8)
    qc.initialize(state, range(8))
    bob(qc, to_alice, to_bob)
    qc.measure_all()
    
    job = execute(qc, backend, memory=True, shots=1)
    result = job.result()
    
    # Works for a noisy simulation
#     counts = result.get_counts()
#     measured_binary = Counter(counts).most_common(1)[0][0]
    
    # Works for a noiseless simulation
    measurements = result.get_memory()
    measured_binary = measurements[0]
    
    reconstructed_crumbs = [''.join([measured_binary[7-qidx] for qidx in tup]) for tup in permutation]
    reconstructed_byte = ''.join(reconstructed_crumbs)
    
    received_char = chr(int(reconstructed_byte, 2))
    decoded.append(received_char)


In [ ]:
print(''.join(decoded))